# Running a job

## Openfoam

We will run an example job using the OpenFoam 7 module for 

`OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7`

Lets load the module:
`module load OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7`

Source the OpenFoam bashrc:

```bash
source $foamDotFile```

Lets switch to our scratch space:

```bash
cd $SCRATCH
```


And copy over the run files:
``` bash
cp -r $WM_PROJECT_DIR/tutorials/multiphase/interFoam/laminar/damBreak ./```

An example run script:
    
```bash
#!/bin/bash
#SBATCH --account=prj3_phase1
#SBATCH --job-name=AMD
#SBATCH --output=AMD.%j.o
#SBATCH --error=AMD.%j.e
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=16
#SBATCH --partition=spot-hbv3-16
#SBATCH --time=04:00:00

module purge
module load OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/hbv3
. $foamDotFile
./Allrun
```

Lets switch into the folder 
```bash
cd damBreak
```

And after copying the above runscript into `run_job.slm` lets submit the job:

```bash
sbatch run_job.slm
```